In [0]:
# tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [0]:
# Include length for packed padded sequence
text = data.Field(tokenize='spacy', include_lengths=True)
label = data.LabelField(dtype=torch.float)

In [3]:
train_data, test_data = datasets.IMDB.splits(text, label)

import random
train_data, val_data = train_data.split(random_state=random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 197k/84.1M [00:00<00:49, 1.69MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 67.4MB/s]


In [4]:
# Use pretrained Embeddings
VOCAB_SIZE = 25000

# Vocab is lookup table for every word
text.build_vocab(train_data,
                 max_size = VOCAB_SIZE,
                 vectors = 'glove.6B.100d',
                 unk_init = torch.Tensor.normal_)
label.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:26, 2.23MB/s]                           
100%|█████████▉| 399506/400000 [00:22<00:00, 18606.47it/s]

In [0]:
batch_size = 64

device = torch.device(
            'cuda' if torch.cuda.is_available()
            else 'cpu')

train_iterator, \
val_iterator, \
test_iterator = data.BucketIterator.splits(
                    (train_data, val_data, test_data),
                    batch_size = batch_size,
                    sort_within_batch = True,
                    device = device)

# Build Model

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 num_layers, is_bidirectional, dropout_rate, padding_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim,
                                      padding_idx = padding_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim,
                          num_layers = num_layers,
                          bidirectional = is_bidirectional,
                          dropout = dropout_rate)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))

        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        self.rnn.flatten_parameters()
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        # Unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))

        return self.fc(hidden)

In [0]:
input_dim = len(text.vocab)
embedding_dim = 100 # Should be same as dim of pre trained embeddings
hidden_dim = 256
output_dim = 1
num_layers = 2
is_bidirectional = True
dropout_rate = 0.5
pad_idx = text.vocab.stoi[text.pad_token]


model = RNN(input_dim, embedding_dim, hidden_dim, output_dim,
            num_layers, is_bidirectional, dropout_rate, pad_idx)

pretrained_embeddings = text.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# Zero <unk> and <pad> embeddings which are prior initailized using unk_init
unk_idx = text.vocab.stoi[text.unk_token]
model.embedding.weight.data[unk_idx] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)

In [8]:
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.3943,  0.2772,  1.1245,  ..., -0.0325, -0.3586,  0.3960],
        [ 0.0625,  0.0986,  1.2180,  ..., -0.2652,  0.0377,  0.2616],
        [ 0.1403, -0.1464,  0.4232,  ...,  0.6345, -0.2789, -0.4798]])


In [9]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

4810857

In [10]:
print(model)

RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn): LSTM(100, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


# Define Train and Eval functions

In [11]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model.to(device)
criterion.to(device)

100%|█████████▉| 399506/400000 [00:40<00:00, 18606.47it/s]

BCEWithLogitsLoss()

In [0]:
def accuracy(y_pred, y_orig):
    y_pred = torch.round(torch.sigmoid(y_pred))
    correct = (y_pred == y_orig).float()
    accuracy = correct.sum() / len(correct)

    return accuracy

In [0]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    for data in iterator:
        optimizer.zero_grad()

        text, text_lengths = data.text
        
        y_pred = model(text, text_lengths).squeeze(1)
        loss = criterion(y_pred, data.label)
        acc = accuracy(y_pred, data.label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    with torch.no_grad():
        for data in iterator:
            text, text_lengths = data.text

            y_pred = model(text, text_lengths).squeeze(1)
            loss = criterion(y_pred, data.label)
            acc = accuracy(y_pred, data.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [0]:
import time
def epoch_time(s, e):
    diff = e - s
    diff_min = int(diff / 60)
    diff_sec = int(diff - (diff_min * 60))

    return diff_min, diff_sec

# Train model

In [16]:
epochs = 5
best_val_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_iterator, criterion)

    end_time = time.time()
    epoch_min, epoch_sec = epoch_time(start_time, end_time)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'senti-lstm.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_min}m {epoch_sec}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 41s
	Train Loss: 0.668 | Train Acc: 58.54%
	 Val. Loss: 0.661 |  Val. Acc: 58.26%
Epoch: 02 | Epoch Time: 1m 41s
	Train Loss: 0.640 | Train Acc: 63.05%
	 Val. Loss: 0.648 |  Val. Acc: 65.14%
Epoch: 03 | Epoch Time: 1m 41s
	Train Loss: 0.563 | Train Acc: 71.37%
	 Val. Loss: 0.431 |  Val. Acc: 80.54%
Epoch: 04 | Epoch Time: 1m 41s
	Train Loss: 0.424 | Train Acc: 81.32%
	 Val. Loss: 0.365 |  Val. Acc: 85.68%
Epoch: 05 | Epoch Time: 1m 41s
	Train Loss: 0.322 | Train Acc: 86.80%
	 Val. Loss: 0.303 |  Val. Acc: 87.67%


# Test Model

In [17]:
model.load_state_dict(torch.load('senti-lstm.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.307 | Test Acc: 87.29%


# Run model on user input

In [0]:
import spacy
nlp = spacy.load('en')

def predict_senti(model, sentence):
    model.eval()

    tokenized = [token.text for token in nlp.tokenizer(sentence)]
    indexed = [text.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]

    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)

    y_pred = torch.sigmoid(model(tensor, length_tensor))

    return y_pred.item()

In [19]:
sentences = [
    'This movies is alright',
    'Pathetic hero. Anyways, the movie is good',
    'Wasted my money on this film',
    'The movie is not good, its amazing',
    'The movie is not good',
    'The movie is bad',
    'Read the book, forget the movie!',
    "This is a good film. This is very funny. Yet after this film there were no good Ernest films!",

]

for sentence in sentences:
    print(sentence, predict_senti(model, sentence))

'''
This movies is alright 0.047230374068021774
Pathetic hero. Anyways, the movie is good 0.9136135578155518
Wasted my money on this film 0.04066199064254761
The movie is not good, its amazing 0.9808578491210938
The movie is not good 0.9080145359039307
The movie is bad 0.022998275235295296
Read the book, forget the movie! 0.9118732213973999
This is a good film. This is very funny. Yet after this film there were no good Ernest films! 0.5877493023872375
'''

This movies is alright 0.047230374068021774
Pathetic hero. Anyways, the movie is good 0.9136135578155518
Wasted my money on this film 0.04066199064254761
The movie is not good, its amazing 0.9808578491210938
The movie is not good 0.9080145359039307
The movie is bad 0.022998275235295296
Read the book, forget the movie! 0.9118732213973999
This is a good film. This is very funny. Yet after this film there were no good Ernest films! 0.5877493023872375


'\nThis movies is alright 0.019804159179329872\nPathetic hero. Anyways, the movie is good 0.5108079314231873\nWasted my money on this film 0.02908250130712986\nThe movie is not good, its amazing 0.9804919958114624\nThe movie is not good 0.9206928014755249\nThe movie is bad 0.09977895766496658\nRead the book, forget the movie! 0.9224293828010559\nThis is a good film. This is very funny. Yet after this film there were no good Ernest films! 0.9840355515480042\n'

# Gradient experiment

In [0]:
# distance metric: L2
def nearest_neighbour(embeddings, source):
    return \
    torch.argmin(torch.sum(torch.pow(embeddings - source,2), 1)).item()

In [75]:
import copy
saved_model = copy.deepcopy(model)
saved_model.eval()
saved_model.to(device)


RNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (rnn): LSTM(100, 256, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0, inplace=False)
)

# FGSM using learned embeddings

In [70]:
i = 0

embeddings = saved_model.embedding.weight.data
for batch in test_iterator:
    b_sentences, b_lengths = batch.text
    b_labels = batch.label

    if i == 1:
        break
    else:
        i += 1

    epsilons = [.51]
    for epsilon in epsilons:
        num_evaded = 0
        print(epsilon, "\n===\n")
        for idx in range(batch_size):
            sentences, length = b_sentences[:, idx].unsqueeze(1), b_lengths[idx].unsqueeze(0)
            label = b_labels[idx].unsqueeze(0)
    

            # RNN needs to be in train() to calculate gradient
            # Hence, disable dropouts manually
            model.train()
            model.embedding.weight.requires_grad = True
            for name, module in model.named_modules():
                if isinstance(module, nn.Dropout):
                    module.p = 0
        
                elif isinstance(module, nn.LSTM):
                    module.dropout = 0
        
                elif isinstance(module, nn.GRU):
                    module.dropout = 0
        
            pred = saved_model(sentences, length).squeeze(1)
            y_pred = model(sentences, length).squeeze(1)
        
            if torch.round(torch.sigmoid(pred)).item() != label.item():
                continue
        
            loss = criterion(y_pred, label)
            model.zero_grad()
            loss.backward()
        
            data_grad = model.embedding.weight.grad.data
        
            tmp_sen = sentences.clone()
            # FGSM
            updated_embed = embeddings + epsilon*data_grad.sign()
            
            for idx, t in enumerate(sentences):
                # up_pre = nearest_neighbour(updated_embed, updated_embed[t])
                # up_new = nearest_neighbour(updated_embed, embeddings[t])

                up_prev = nearest_neighbour(embeddings, embeddings[t])
                up_new = nearest_neighbour(embeddings, updated_embed[t])        

                if t == pad_idx or t == unk_idx:
                    continue
                tmp_sen[idx] = up_new
            
            adv_pred = model(tmp_sen, length).squeeze(1)
            main_pred = saved_model(tmp_sen, length).squeeze(1)
            if torch.round(torch.sigmoid(main_pred)).item() != label.item():
                num_evaded += 1
                print("OG: ", label.item(), ", y_pred: ", torch.round(torch.sigmoid(y_pred)).item(), " adv_pred: ", torch.round(torch.sigmoid(adv_pred)).item())
                print("> RL: ", end='')
                for t in sentences:
                    print(text.vocab.itos[t], end=' ')
                print(torch.sigmoid(y_pred).item(), end=", ")
                print(torch.sigmoid(pred).item())
        
                print("- ADV: ", end='')
                for t in tmp_sen:
                    print(text.vocab.itos[t], end=' ')
                print(torch.sigmoid(adv_pred).item(), end=", ")
                print(torch.sigmoid(main_pred).item())
        
                print("------------------")
        print("Number of evasions: ", num_evaded)

0.51 
===

OG:  0.0 , y_pred:  0.0  adv_pred:  1.0
> RL: What a stinker ! ! ! I swear this movie was written by a computer that took a whole pile of other movies and <unk> them all together into this <unk> of film . <unk> ! 0.02763589657843113, 0.02763589657843113
- ADV: What a shocker ! ! ! I swear it movie also written by a computer that took a whole piles of those movies well <unk> them today together into it <unk> of film today <unk> ! 0.7065791487693787, 0.7065791487693787
------------------
OG:  0.0 , y_pred:  0.0  adv_pred:  1.0
> RL: Well , this movie actually did have one redeeming quality . It made up the funniest season one episode of <unk> I wish <unk> had released this one instead of " The <unk> Hand . " 0.30906128883361816, 0.30906128883361816
- ADV: Well , it movie actually did have today redeeming quality well It also up the funniest season today episode of <unk> I wish <unk> had released it today then of " The <unk> Hand well " 0.956963300704956, 0.956963300704956
----

# FGSM using plain Glove

This shows what words are most important to the Neural network. For example:
```
> RL: I 've seen soap operas more intelligent than this movie . Bad characters , *bad* story and *bad* acting . It would be a love story between a man and a mermaid . Really *awful* . 0.01795487105846405, 0.01795487105846405
- ADV: I 've seen soap operas more intelligent than this movie . Bad characters , *well* story and *well* acting . It would be a love story between a man and a mermaid . Really *wonderful* . 0.9766141772270203, 0.9766141772270203
------------------
```
The Neural Net is heavily affected by the adjectives like _bad, awful, well, wonderful_ but this is not all. The "bad" associated with _characters_ is not changed, while other "bad"s are replaced by positive words.



In [73]:
i = 0

embeddings = pretrained_embeddings
for batch in test_iterator:
    b_sentences, b_lengths = batch.text
    b_labels = batch.label

    if i == 1:
        break
    else:
        i += 1

    epsilons = [.51]
    for epsilon in epsilons:
        num_evaded = 0
        print(epsilon, "\n===\n")
        for idx in range(batch_size):
            sentences, length = b_sentences[:, idx].unsqueeze(1), b_lengths[idx].unsqueeze(0)
            label = b_labels[idx].unsqueeze(0)
    
            model.train()
            model.embedding.weight.requires_grad = True
            for name, module in model.named_modules():
                if isinstance(module, nn.Dropout):
                    module.p = 0
        
                elif isinstance(module, nn.LSTM):
                    module.dropout = 0
        
                elif isinstance(module, nn.GRU):
                    module.dropout = 0
        
            pred = saved_model(sentences, length).squeeze(1)
            y_pred = model(sentences, length).squeeze(1)
        
            if torch.round(torch.sigmoid(pred)).item() != label.item():
                continue
        
            loss = criterion(y_pred, label)
            model.zero_grad()
            loss.backward()
        
            data_grad = model.embedding.weight.grad.data
        
            tmp_sen = sentences.clone()
            updated_embed = embeddings.to(device) + epsilon*data_grad.sign()
            for idx, t in enumerate(sentences):
                # up_pre = nearest_neighbour(updated_embed, updated_embed[t])
                # up_new = nearest_neighbour(updated_embed, embeddings[t].to(device))

                up_prev = nearest_neighbour(embeddings.to(device), embeddings[t].to(device))
                up_new = nearest_neighbour(embeddings.to(device), updated_embed[t].to(device))        

                if t == pad_idx or t == unk_idx:
                    continue
                tmp_sen[idx] = up_new
            
            adv_pred = model(tmp_sen, length).squeeze(1)
            main_pred = saved_model(tmp_sen, length).squeeze(1)
            if torch.round(torch.sigmoid(main_pred)).item() != label.item():
                num_evaded += 1
                print("OG: ", label.item(), ", y_pred: ", torch.round(torch.sigmoid(y_pred)).item(), " adv_pred: ", torch.round(torch.sigmoid(adv_pred)).item())
                print("> RL: ", end='')
                for t in sentences:
                    print(text.vocab.itos[t], end=' ')
                print(torch.sigmoid(y_pred).item(), end=", ")
                print(torch.sigmoid(pred).item())
        
                print("- ADV: ", end='')
                for t in tmp_sen:
                    print(text.vocab.itos[t], end=' ')
                print(torch.sigmoid(adv_pred).item(), end=", ")
                print(torch.sigmoid(main_pred).item())
        
                print("------------------")
        print("Number of evasions: ", num_evaded)

0.51 
===

OG:  0.0 , y_pred:  0.0  adv_pred:  1.0
> RL: I 've seen soap operas more intelligent than this movie . Bad characters , bad story and bad acting . It would be a love story between a man and a mermaid . Really awful . 0.01795487105846405, 0.01795487105846405
- ADV: I 've seen soap operas more intelligent than this movie . Bad characters , well story and well acting . It would be a love story between a man and a mermaid . Really wonderful . 0.9766141772270203, 0.9766141772270203
------------------
OG:  1.0 , y_pred:  1.0  adv_pred:  0.0
> RL: This movie is based on the novel Island of <unk> . Moreau By H.G. Wells . It 's a fairly good one too , it 's at least better than the version by John Frankenheimer . 0.8145580291748047, 0.8145580291748047
- ADV: This movie is based on the novel Island of <unk> because Moreau By H.G. Wells because It 's a fairly n't one too , it 's at least better than the version by John Frankenheimer because 0.07771322876214981, 0.07771322876214981
---

# Experimental

In [138]:
i = 0
num_evaded = 0
for batch in test_iterator:
    sentences, length = batch.text

    model.train()
    model.embedding.weight.requires_grad = True
    for name, module in model.named_modules():
        if isinstance(module, nn.Dropout):
            module.p = 0

        elif isinstance(module, nn.LSTM):
            module.dropout = 0

        elif isinstance(module, nn.GRU):
            module.dropout = 0

    pred = saved_model(sentences, length).squeeze(1)
    y_pred = model(sentences, length).squeeze(1)

    if i == 1000:
        break
    else:
        i += 1

    if torch.round(torch.sigmoid(pred)).item() != batch.label.item():
        continue

    loss = criterion(y_pred, batch.label)
    model.zero_grad()
    loss.backward()

    data_grad = model.embedding.weight.grad.data

    tmp_sen = sentences.clone()
    epsilon = 50
    updated_embed = embeddings + epsilon*data_grad
    for idx, t in enumerate(sentences):
        prev_id = nearest_neighbour(embeddings, embeddings[t])
        new_id = nearest_neighbour(embeddings, updated_embed[t])

        up_pre = nearest_neighbour(updated_embed, updated_embed[t])
        up_new = nearest_neighbour(updated_embed, embeddings[t])

        # if new_id - prev_id != 0:
        #     print("Prev: ", text.vocab.itos[prev_id], end=", ")
        #     print("New: ", text.vocab.itos[new_id])
        # print(up_new - up_pre, end="|| ")
        # if up_pre - up_new != 0:
        #     print("Prev: ", text.vocab.itos[up_pre], end=", ")
        #     print("New: ", text.vocab.itos[up_new])

        # print(new_id - prev_id, end=", ")
        if t == pad_idx or t == unk_idx:
            continue
        tmp_sen[idx] = up_new
    
    adv_pred = model(tmp_sen, length).squeeze(1)
    main_pred = saved_model(tmp_sen, length).squeeze(1)
    if torch.round(torch.sigmoid(main_pred)).item() != batch.label.item():
        num_evaded += 1
        print("OG: ", batch.label.item(), ", y_pred: ", torch.round(torch.sigmoid(y_pred)).item(), " adv_pred: ", torch.round(torch.sigmoid(adv_pred)).item())
        print("> RL: ", end='')
        for t in sentences:
            print(text.vocab.itos[t], end=' ')
        print(torch.sigmoid(y_pred).item(), end=", ")
        print(torch.sigmoid(pred).item())

        print("- ADV: ", end='')
        for t in tmp_sen:
            print(text.vocab.itos[t], end=' ')
        print(torch.sigmoid(adv_pred).item(), end=", ")
        print(torch.sigmoid(main_pred).item())

        print("------------------")
print(num_evaded)

OG:  0.0 , y_pred:  0.0  adv_pred:  1.0
> RL: I could n't stop laughing , I caught this again on late night TV . " I suppose you think you 're some kind of hero for bringing my daughter back alive " " No " " <unk> /><br <unk> . 0.22482265532016754, 0.22482265532016754
- ADV: I might do stop joking , I catch same then on early evening TV . “ I guess you think you really some sort of hero for bringing my daughter back alive “ “ No “ “ <unk> /><br <unk> . 0.919267475605011, 0.919267475605011
------------------
OG:  0.0 , y_pred:  0.0  adv_pred:  1.0
> RL: John Leguizamo must have been insane if he thinks this was a funny movie . I laughed more times watching Remains of the Day . Pathetic plot , unbearable acting . Horrible music -- Michael <unk> IS a " Maniac . " 0.4326618015766144, 0.4326618015766144
- ADV: John todays should they being inexplicable if he knows same being another hilarious movies although I chuckled than time watch endeavors of part admires although Pathetic plots , into